In [ ]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import math
import numpy as np
import scipy
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
import matplotlib.pyplot as plt
import logging
import time
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from torch.cuda.amp import autocast
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [ ]:
from datasets import load_dataset
from datasets import load_from_disk


In [ ]:
metadata_dataset = load_from_disk("/content/drive/MyDrive/Datasets/Sumit_datasets/dataset_cleaned_1000_products/hf_meta_1300_products")
reviews_dataset = load_from_disk("/content/drive/MyDrive/Datasets/Sumit_datasets/dataset_cleaned_1000_products/hf_reviews_20_each")

In [ ]:
metadata_dataset

Dataset({
    features: ['parent_asin', 'category3', 'title', 'description', 'average_rating'],
    num_rows: 26000
})

In [ ]:
reviews_dataset

Dataset({
    features: ['parent_asin', 'text', 'helpful_vote', 'rating', 'images', 'category'],
    num_rows: 463555
})

In [ ]:
metadata_df = metadata_dataset.to_pandas()
reviews_df = reviews_dataset.to_pandas()

In [ ]:
stopwords.words('english')
nltk.download('stopwords')
stemmer = PorterStemmer()
pattern = r'[\[\]<>{}.,:;?\-!\"\'`~@#$%^&*()_+=\\|/]'
HTML_TAG_REGEX = re.compile(r'<[^>]+>')
STOP_WORDS = set(stopwords.words('english'))
emoji_pattern = re.compile(
    "["
    "\U0001F600-\U0001F64F"
    "\U0001F300-\U0001F5FF"
    "\U0001F680-\U0001F6FF"
    "\U0001F1E0-\U0001F1FF"
    "\U00002702-\U000027B0"
    "\U000024C2-\U0001F251"
    "\U0001FA70-\U0001FAFF"
    "\U00002600-\U000026FF"
    "\U00002B50"
    "\U0000FE0F"
    "]+",
    flags=re.UNICODE
)

def preprocess_text(text):
    if not isinstance(text, str):
        return ''
    text = HTML_TAG_REGEX.sub(' ', text)
    text = emoji_pattern.sub(' ', text)
    text = text.lower()
    text = re.sub(pattern, ' ', text)
    words = text.split()
    filtered_words = [word for word in words if word not in STOP_WORDS and len(word) > 1]
    text = ' '.join(filtered_words)
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
metadata_df['description'] = metadata_df['description'].astype(str)

In [ ]:
metadata_df['processed_title'] = metadata_df['title'].apply(preprocess_text)

In [ ]:
metadata_df['processed_description'] = metadata_df['description'].apply(preprocess_text)

In [ ]:
metadata_df.head()

,parent_asin,category3,title,description,average_rating,processed_title,processed_description
0,B0002H4YUI,Adapters,Kensington K33117 International All-in-One Tra...,['Product description'\n 'International Travel...,4.3,kensington k33117 international one travel plu...,product description international travel plug ...
1,B002RWJD84,Adapters,Alphasonik APA100 Worldwide Universal Internat...,"['Product Description'\n ""When you're travelin...",4.2,alphasonik apa100 worldwide universal internat...,product description traveling overseas good tr...
2,B00FB50S0Q,Adapters,Amazon Basics 4 Port USB to USB 3.0 Hub with 5...,['Product Description'\n 'Amazon Basics 4 Port...,4.5,amazon basics port usb usb hub 5v 5a power ada...,product description amazon basics port usb usb...
3,B007SJGGAE,Adapters,Anker® Uspeed PCI-E to USB 3.0 2 Port Express ...,['Introduction'\n 'With quick and easy install...,4.2,anker® uspeed pci usb port express card usb 20...,introduction quick easy installation uspeed us...
4,B0000511OJ,Adapters,Olympus C7AU AC Adapter for Olympus Cameras,['Product description'\n 'Olympus America (200...,4.0,olympus c7au ac adapter olympus cameras,product description olympus america 200685 ac ...


In [ ]:
reviews_df['text'] = reviews_df['text'].astype(str)

In [ ]:
reviews_df['text_processed'] = reviews_df['text'].apply(preprocess_text)

In [ ]:
metadata_df['combined_text'] = metadata_df['title'] + ' ' + metadata_df['description']

In [ ]:
metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26000 entries, 0 to 25999
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   parent_asin            26000 non-null  object 
 1   category3              26000 non-null  object 
 2   title                  26000 non-null  object 
 3   description            26000 non-null  object 
 4   average_rating         26000 non-null  float64
 5   processed_title        26000 non-null  object 
 6   processed_description  26000 non-null  object 
 7   combined_text          26000 non-null  object 
dtypes: float64(1), object(7)
memory usage: 1.6+ MB


In [ ]:
metadata_df.head()

,parent_asin,category3,title,description,average_rating,processed_title,processed_description,combined_text
0,B0002H4YUI,Adapters,Kensington K33117 International All-in-One Tra...,['Product description'\n 'International Travel...,4.3,kensington k33117 international one travel plu...,product description international travel plug ...,Kensington K33117 International All-in-One Tra...
1,B002RWJD84,Adapters,Alphasonik APA100 Worldwide Universal Internat...,"['Product Description'\n ""When you're travelin...",4.2,alphasonik apa100 worldwide universal internat...,product description traveling overseas good tr...,Alphasonik APA100 Worldwide Universal Internat...
2,B00FB50S0Q,Adapters,Amazon Basics 4 Port USB to USB 3.0 Hub with 5...,['Product Description'\n 'Amazon Basics 4 Port...,4.5,amazon basics port usb usb hub 5v 5a power ada...,product description amazon basics port usb usb...,Amazon Basics 4 Port USB to USB 3.0 Hub with 5...
3,B007SJGGAE,Adapters,Anker® Uspeed PCI-E to USB 3.0 2 Port Express ...,['Introduction'\n 'With quick and easy install...,4.2,anker® uspeed pci usb port express card usb 20...,introduction quick easy installation uspeed us...,Anker® Uspeed PCI-E to USB 3.0 2 Port Express ...
4,B0000511OJ,Adapters,Olympus C7AU AC Adapter for Olympus Cameras,['Product description'\n 'Olympus America (200...,4.0,olympus c7au ac adapter olympus cameras,product description olympus america 200685 ac ...,Olympus C7AU AC Adapter for Olympus Cameras ['...


In [ ]:
reviews_df.head()

,parent_asin,text,helpful_vote,rating,images,category,text_processed
0,0062970704,5 burning gas balls,527,5.0,[],Blacks,burning gas balls
1,0062970704,Strange Planet revolves around a simple concei...,205,5.0,[],Blacks,strange planet revolves around simple conceit ...
2,0062970704,WARNING: MIGHT INCLUDE SPOILER; READ THIS REVI...,129,5.0,"[{'attachment_type': 'IMAGE', 'large_image_url...",Blacks,warning might include spoiler read review risk...
3,0062970704,"When I was a kid, I had 2 favorite comic serie...",64,5.0,[],Blacks,kid favorite comic series every book garfield ...
4,0062970704,It’s a great book obviously. But it arrived in...,7,3.0,"[{'attachment_type': 'IMAGE', 'large_image_url...",Blacks,it’s great book obviously arrived large box pr...


In [ ]:
reviews_agg = (
    reviews_df
    .groupby('parent_asin')['text_processed']
    .apply(lambda texts: ' '.join(texts))
    .reset_index()
    .rename(columns={'text_processed': 'reviews'})
)

In [ ]:
reviews_agg.head()

,parent_asin,reviews
0,0062970704,burning gas balls strange planet revolves arou...
1,0071341196,randy slone regurgitates one states words clea...
2,0140194614,quot myths live quot one books makes wish assi...
3,0151004714,best novel read years everything fiction beaut...
4,0345803507,figured might well title review something said...


In [ ]:
review_meta_df = pd.merge(metadata_df, reviews_agg, on='parent_asin', how='inner')

In [ ]:
review_meta_df.rename(columns={'category3': 'category'}, inplace=True)

In [ ]:
review_meta_df.head()

,parent_asin,category,title,description,average_rating,processed_title,processed_description,combined_text,reviews
0,B0002H4YUI,Adapters,Kensington K33117 International All-in-One Tra...,['Product description'\n 'International Travel...,4.3,kensington k33117 international one travel plu...,product description international travel plug ...,Kensington K33117 International All-in-One Tra...,update jan 2014 several people said able use a...
1,B002RWJD84,Adapters,Alphasonik APA100 Worldwide Universal Internat...,"['Product Description'\n ""When you're travelin...",4.2,alphasonik apa100 worldwide universal internat...,product description traveling overseas good tr...,Alphasonik APA100 Worldwide Universal Internat...,purchased j2 earbuds serious flying beginning ...
2,B00FB50S0Q,Adapters,Amazon Basics 4 Port USB to USB 3.0 Hub with 5...,['Product Description'\n 'Amazon Basics 4 Port...,4.5,amazon basics port usb usb hub 5v 5a power ada...,product description amazon basics port usb usb...,Amazon Basics 4 Port USB to USB 3.0 Hub with 5...,love amazon hate one star one products encount...
3,B007SJGGAE,Adapters,Anker® Uspeed PCI-E to USB 3.0 2 Port Express ...,['Introduction'\n 'With quick and easy install...,4.2,anker® uspeed pci usb port express card usb 20...,introduction quick easy installation uspeed us...,Anker® Uspeed PCI-E to USB 3.0 2 Port Express ...,bought uspeed usb pci card amazon order latest...
4,B0000511OJ,Adapters,Olympus C7AU AC Adapter for Olympus Cameras,['Product description'\n 'Olympus America (200...,4.0,olympus c7au ac adapter olympus cameras,product description olympus america 200685 ac ...,Olympus C7AU AC Adapter for Olympus Cameras ['...,little hesitant purchase ac adapter descriptio...


In [ ]:
review_meta_df['reviews'] = review_meta_df['reviews'].fillna('')

In [ ]:
review_meta_df['combined_text']= review_meta_df['combined_text'].fillna('')

In [ ]:
specific_product_data = review_meta_df[review_meta_df['parent_asin'] == 'B00A3RVNXI']
specific_product_data.head()

,parent_asin,category,title,description,average_rating,processed_title,processed_description,combined_text,reviews
11076,B00A3RVNXI,Headphones,Harman Kardon CL Precision On-Ear Headphones w...,['Enjoy high-performance sound quality with a ...,3.9,harman kardon cl precision ear headphones exte...,enjoy high performance sound quality sophistic...,Harman Kardon CL Precision On-Ear Headphones w...,like aesthetics presentation packaging beautif...


In [ ]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("GPU found. Using GPU.")
    else:
        device = torch.device("cpu")
        print("GPU not found. Using CPU.")
    return device

In [ ]:
def load_model(device):
    model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
    return model

In [ ]:
metadata_to_embed = review_meta_df['combined_text'].tolist()
reviews_to_embed = review_meta_df['reviews'].tolist()
asins_in_order = review_meta_df['parent_asin'].tolist()

In [ ]:
model = load_model(get_device())

GPU found. Using GPU.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def create_meta_embeddings():
  if metadata_to_embed:
    print(f"\nGenerating embeddings for {len(metadata_to_embed)} product texts...")
    m_embeddings = model.encode(metadata_to_embed, show_progress_bar=True, convert_to_tensor=True)
    print(f"Embeddings generated. Shape: {m_embeddings.shape}") # Should be (num_products, 384) for MiniLM-L6-v2
    first_asin = asins_in_order[0]
    print(f"Embedding for first product: {m_embeddings[asins_in_order.index(first_asin)]}")
    return m_embeddings

In [ ]:
def create_review_embeddings():
  if reviews_to_embed:
    print(f"\nGenerating embeddings for {len(reviews_to_embed)} product texts...")

    r_embeddings = model.encode(reviews_to_embed, show_progress_bar=True, convert_to_tensor=True)

    print(f"Embeddings generated. Shape: {r_embeddings.shape}") # Should be (num_products, 384) for MiniLM-L6-v2
    first_asin = asins_in_order[0]
    print(f"Embedding for first product: {r_embeddings[asins_in_order.index(first_asin)]}")
    return r_embeddings

In [ ]:
review_embeddings = create_review_embeddings()
review_asin_map= dict(zip(asins_in_order, review_embeddings))
metadata_embeddings = create_meta_embeddings()
metadata_asin_map = dict(zip(asins_in_order, metadata_embeddings))


Generating embeddings for 26000 product texts...


Batches:   0%|          | 0/813 [00:00<?, ?it/s]

Embeddings generated. Shape: torch.Size([26000, 384])
Embedding for first product: tensor([-3.8687e-02, -2.5794e-02,  8.7318e-03,  2.6043e-03,  4.7004e-02,
         2.1161e-02,  6.1280e-02,  2.1913e-02, -7.0999e-02,  3.2303e-02,
         1.0186e-01,  1.9512e-02,  6.9156e-02,  5.2620e-02,  2.2070e-02,
         3.4481e-02,  1.6483e-02, -1.6983e-02, -6.9565e-02,  3.1268e-02,
         1.8621e-02,  1.6872e-02, -6.4933e-02,  1.2067e-02, -4.9006e-02,
         1.0295e-02, -1.5735e-02, -1.7780e-02, -7.1487e-02, -6.7426e-02,
        -7.0171e-02,  5.8312e-02, -4.0603e-02, -1.7514e-02,  5.6287e-02,
        -5.4714e-02,  4.1260e-03,  3.7862e-02, -2.8579e-02, -1.3151e-01,
         1.2316e-02, -1.3879e-01,  1.0375e-03, -1.2312e-02,  2.7442e-02,
        -5.7021e-02, -6.2746e-03, -6.9874e-02,  3.5196e-02, -8.6345e-02,
        -9.4037e-03, -2.9001e-02,  5.8396e-02,  2.4388e-02, -4.8736e-02,
        -8.8649e-02,  5.3188e-02,  2.0543e-02, -7.6486e-03, -8.2998e-02,
        -5.5365e-03, -7.6137e-02,  1.5006

Batches:   0%|          | 0/813 [00:00<?, ?it/s]

Embeddings generated. Shape: torch.Size([26000, 384])
Embedding for first product: tensor([-3.1181e-02,  7.1855e-02,  5.1984e-02,  6.1057e-03,  3.7149e-02,
         2.3386e-02, -1.9834e-02, -2.2946e-02,  2.3344e-02,  2.0077e-03,
         7.0209e-02, -4.4787e-02, -2.2990e-02,  5.3594e-02, -3.6505e-02,
         2.3359e-02,  5.5821e-02, -3.5901e-03, -5.7629e-02, -3.3120e-04,
         6.4910e-02, -7.6858e-03,  1.2626e-02, -2.4524e-02,  2.3590e-03,
         6.3893e-02,  8.8976e-02, -2.6787e-02, -1.0195e-01, -3.1552e-02,
        -4.2552e-02, -1.0063e-01, -3.4077e-02, -4.4727e-03,  1.4412e-02,
        -2.4215e-02, -4.1246e-03,  1.4802e-02, -5.2600e-03, -2.8704e-02,
        -7.0790e-02, -5.4224e-02,  6.8221e-02,  2.1029e-02, -2.1772e-02,
        -2.9911e-02, -2.8411e-02, -2.3251e-02, -9.7621e-03, -4.1540e-02,
         1.0830e-01, -2.4137e-03, -2.9646e-02,  1.8361e-02, -1.5646e-03,
        -4.2394e-02, -4.8086e-03,  2.4034e-02,  8.0402e-02, -1.1978e-02,
         2.9311e-02, -3.2580e-02,  1.8798

In [ ]:
len(review_embeddings)

26000

In [ ]:
len(metadata_embeddings)

26000

In [ ]:
from typing import List, Sequence
def valtidate_embeddings(embeddings, asins: Sequence[str]):
    if embeddings is None or len(embeddings) == 0:
        raise ValueError("Product embeddings are empty.")
    if len(embeddings) != len(asins):
        raise ValueError(
            "Mismatch between number of embeddings "
            f"({len(embeddings)}) and number of ASINs ({len(asins)})."
        )

In [ ]:
def create_query_embedding(query: str, model: SentenceTransformer):
    print(f"Embedding user query: “{query}” …")
    process_query = preprocess_text(query)
    return model.encode(process_query, show_progress_bar=True, convert_to_tensor=True)


In [ ]:
def retrieve_top_hits(
    query_embedding,
    corpus_embeddings,
    top_k: int = 50
):
    """Run semantic search and return the raw hit list from SBERT utilities."""
    print("Calculating similarities …")
    return util.semantic_search(query_embedding, corpus_embeddings, top_k=top_k)[0]

In [ ]:
def hits_to_dataframe(
    hits,
    asins: Sequence[str]
) -> pd.DataFrame:
    """
    Convert SBERT hit dicts → tidy DataFrame with ASINs and similarity scores.
    """
    print(f"Found {len(hits)} similar products.")
    rows = [
        {"parent_asin": asins[hit["corpus_id"]], "similarity_score": hit["score"]}
        for hit in hits
    ]
    return pd.DataFrame(rows)

In [ ]:
def show_results(df: pd.DataFrame, query: str) -> None:
    """Nicely print the ranked results to stdout."""
    if df.empty:
        print("No similar products found.")
        return

    print(f"\nTop {len(df)} most similar products to “{query}”:\n")
    for idx, (asin, score) in enumerate(df.itertuples(index=False), start=1):
        print(f"{idx:2}. ASIN: {asin} | Score: {score: .4f}")

In [ ]:
def find_similar_products(
    query: str,
    model: SentenceTransformer,
    metadata_embeddings,
    product_asins: Sequence[str],
    top_n: int = 50,
) -> pd.DataFrame:
    """
    High‑level wrapper: validate → embed → search → format.
    Returns a DataFrame ready for downstream use.
    """
    valtidate_embeddings(metadata_embeddings, product_asins)

    query_emb = create_query_embedding(query, model)
    raw_hits = retrieve_top_hits(query_emb, metadata_embeddings, top_k=top_n)
    return hits_to_dataframe(raw_hits, product_asins)


In [ ]:
def get_user_query():
  """Gets a user query from the input field."""
  user_query = input("Enter your product query: ")
  return user_query



In [ ]:
user_query=get_user_query()

Enter your product query: ’m searching for something to listen to music. i stay in a noise place so i dont want outside sound while listening to music.


In [ ]:
from sentence_transformers import util

In [ ]:
df_results = find_similar_products(
            query=user_query,
            model=model,
            metadata_embeddings=metadata_embeddings,
            product_asins=asins_in_order,
            top_n=50,
        )

print(df_results)

Embedding user query: “’m searching for something to listen to music. i stay in a noise place so i dont want outside sound while listening to music.” …


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating similarities …
Found 50 similar products.
   parent_asin  similarity_score
0   B01N6UAWTY          0.503670
1   B01N14AT73          0.503215
2   B00E9262IE          0.494491
3   B01N4ONWV3          0.487851
4   B00WUTHOQ2          0.486593
5   B07D36QM7T          0.486362
6   B00GOOWRR0          0.478011
7   B00E9262YS          0.476288
8   B00WUNE7FE          0.475348
9   B07PXGDCZJ          0.471837
10  B001CK9760          0.458254
11  B001GS8FZ2          0.457298
12  B0015BYKGI          0.457109
13  B001NTDGQC          0.452406
14  B079ND3JM1          0.451778
15  B000BPRGK6          0.451684
16  B00014B4OQ          0.451587
17  B019MBSRQQ          0.443119
18  B00WU35CKS          0.441050
19  B004E0ZLZQ          0.438120
20  B004M8ST5Y          0.437975
21  B00R9DJ1DI          0.437789
22  B00EM7C6VS          0.437489
23  B00AEY4QP2          0.436558
24  B0054JJ0QW          0.435770
25  B0002K6Y9E          0.435048
26  B06XXMFMG8          0.434142
27  B00XJT1RGU        

In [ ]:
# prompt: take the parent_Asin from df_result and find respective title description and reviews from metadata_df and reviews_df respectively and create a final_df .

import pandas as pd
def create_final_df(df_results, metadata_df, reviews_df):
    """
    Merges data from df_results, metadata_df, and reviews_df based on 'parent_asin'.
    """
    final_df = pd.merge(df_results, metadata_df[['parent_asin', 'title', 'description']], on='parent_asin', how='inner')
    final_df = pd.merge(final_df, reviews_df[['parent_asin', 'text']], on='parent_asin', how='inner')
    return final_df

final_df = create_final_df(df_results, metadata_df, reviews_df)
final_df


,parent_asin,similarity_score,title,description,text
0,B01N6UAWTY,0.503670,Libratone Lightning in-Ear Active Noise Cancel...,['Product Description'\n 'Libratone Q Adapt In...,I LOVE THESE HEADPHONES. Recent convert from ...
1,B01N6UAWTY,0.503670,Libratone Lightning in-Ear Active Noise Cancel...,['Product Description'\n 'Libratone Q Adapt In...,Don't expect Bose type of noise cancellations....
2,B01N6UAWTY,0.503670,Libratone Lightning in-Ear Active Noise Cancel...,['Product Description'\n 'Libratone Q Adapt In...,These are my new favorite earbuds. I am not t...
3,B01N6UAWTY,0.503670,Libratone Lightning in-Ear Active Noise Cancel...,['Product Description'\n 'Libratone Q Adapt In...,I have had numerous flights in the last month ...
4,B01N6UAWTY,0.503670,Libratone Lightning in-Ear Active Noise Cancel...,['Product Description'\n 'Libratone Q Adapt In...,Bought two of these based on the reviews and w...
...,...,...,...,...,...
968,B002LARRDA,0.418522,Logitech Squeezebox Touch (Discontinued by Man...,['Product Description'\n 'Logitech Squeezebox ...,I haven't been so excited about a litle gadget...
969,B002LARRDA,0.418522,Logitech Squeezebox Touch (Discontinued by Man...,['Product Description'\n 'Logitech Squeezebox ...,I now have 4 Squeezeboxes. They simply use my...
970,B002LARRDA,0.418522,Logitech Squeezebox Touch (Discontinued by Man...,['Product Description'\n 'Logitech Squeezebox ...,Ordered a Squeezebox Touch yesterday direct fr...
971,B002LARRDA,0.418522,Logitech Squeezebox Touch (Discontinued by Man...,['Product Description'\n 'Logitech Squeezebox ...,The Logitech Squeezebox Touch is a fantastic m...


In [ ]:
'''
model = load_model(get_device())
from sentence_transformers import util

def find_similar_products(query, model, metadata_embeddings, product_asins, top_n=50):
    if metadata_embeddings is None or len(metadata_embeddings) == 0:
        print("Warning: Product embeddings are empty.")
        return []
    if len(metadata_embeddings) != len(product_asins):
        raise ValueError("Mismatch between number of embeddings and number of ASINs.")

    print(f"Embedding the user query: '{query}'")
    query_embedding = model.encode(query, show_progress_bar=True,  convert_to_tensor=True)
    print("Calculating similarities...")


    hits = util.semantic_search(query_embedding, metadata_embeddings, top_k=top_n)
    results_df = pd.DataFrame(columns=['parent_asin', 'similarity_score'])

    if hits and hits[0]:
        print(f"Found {len(hits[0])} similar products.")
        for hit in hits[0]:
            corpus_id = hit['corpus_id']
            score = hit['score']
            asin = product_asins[corpus_id]
            results_df = pd.concat([results_df, pd.DataFrame({'parent_asin': [asin], 'similarity_score': [score]})], ignore_index=True)
    else:
        print("No similar products found based on the query.")
    return results_df


user_query = "I want to buy a powerbank  which i can use for travel"
top_n_results = 50


if 'metadata_embeddings' in locals() and metadata_embeddings is not None and len(metadata_embeddings) > 0:
    similar_products = find_similar_products(
        query=user_query,
        model=model,
        metadata_embeddings=metadata_embeddings,
        product_asins=asins_in_order,
        top_n=top_n_results
    )

    print(f"\nTop {len(similar_products)} most similar products to '{user_query}':")
    if similar_products is not None and len(similar_products) > 0:
      for i, (asin, score) in enumerate(similar_products.values):
            print(f"{i+1}. ASIN: {asin}, Similarity Score: {score:.4f}")
    else:
        print("No results found.")

else:
    print("\nCannot perform search because product embeddings are not available.")
    '''


'\nmodel = load_model(get_device())\nfrom sentence_transformers import util\n\ndef find_similar_products(query, model, metadata_embeddings, product_asins, top_n=50):\n    if metadata_embeddings is None or len(metadata_embeddings) == 0:\n        print("Warning: Product embeddings are empty.")\n        return []\n    if len(metadata_embeddings) != len(product_asins):\n        raise ValueError("Mismatch between number of embeddings and number of ASINs.")\n\n    print(f"Embedding the user query: \'{query}\'")\n    query_embedding = model.encode(query, show_progress_bar=True,  convert_to_tensor=True)\n    print("Calculating similarities...")\n\n\n    hits = util.semantic_search(query_embedding, metadata_embeddings, top_k=top_n)\n    results_df = pd.DataFrame(columns=[\'parent_asin\', \'similarity_score\'])\n\n    if hits and hits[0]:\n        print(f"Found {len(hits[0])} similar products.")\n        for hit in hits[0]:\n            corpus_id = hit[\'corpus_id\']\n            score = hit[\'s

In [ ]:
def semantic_search_similar_products(similar_products_df, query_embedding, review_embeddings, metadata_embeddings):
    results = {}
    for asin in similar_products_df['parent_asin'].unique():
        if asin in review_embeddings and asin in metadata_embeddings:
            alpha = 0.6 # Example: give slightly more weight to metadata
            combined_embedding = (alpha * metadata_embeddings[asin] + (1 - alpha) * review_embeddings[asin])

            # Calculate cosine similarity
            cosine_scores_combined = util.cos_sim(query_embedding, combined_embedding)[0]

            hits = util.semantic_search(query_embedding, [combined_embedding])
            if hits and hits[0]:
                for hit in hits[0]:
                    score = hit['score']
                    results.setdefault(asin, []).append((asin, score))
        else:
            results[asin] = []
    return results
user_query ="’m searching for something to listen to music. i stay in a noise place so i dont want outside sound while listening to music."
query_embedding = model.encode(user_query)
query_embedding = model.encode(user_query, show_progress_bar=True,  convert_to_tensor=True)
search_results = semantic_search_similar_products(df_results, query_embedding, review_asin_map, metadata_asin_map)
print(search_results)




Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'B01N6UAWTY': [('B01N6UAWTY', 0.5133792161941528)], 'B01N14AT73': [('B01N14AT73', 0.4716483950614929)], 'B00E9262IE': [('B00E9262IE', 0.4975065588951111)], 'B01N4ONWV3': [('B01N4ONWV3', 0.458170622587204)], 'B00WUTHOQ2': [('B00WUTHOQ2', 0.4452399015426636)], 'B07D36QM7T': [('B07D36QM7T', 0.4324036240577698)], 'B00GOOWRR0': [('B00GOOWRR0', 0.4548862874507904)], 'B00E9262YS': [('B00E9262YS', 0.47203901410102844)], 'B00WUNE7FE': [('B00WUNE7FE', 0.5193489193916321)], 'B07PXGDCZJ': [('B07PXGDCZJ', 0.4520835876464844)], 'B001CK9760': [('B001CK9760', 0.5246155261993408)], 'B001GS8FZ2': [('B001GS8FZ2', 0.43464457988739014)], 'B0015BYKGI': [('B0015BYKGI', 0.5043649673461914)], 'B001NTDGQC': [('B001NTDGQC', 0.3956969082355499)], 'B079ND3JM1': [('B079ND3JM1', 0.41663658618927)], 'B000BPRGK6': [('B000BPRGK6', 0.43484359979629517)], 'B00014B4OQ': [('B00014B4OQ', 0.444000780582428)], 'B019MBSRQQ': [('B019MBSRQQ', 0.4408206343650818)], 'B00WU35CKS': [('B00WU35CKS', 0.44786566495895386)], 'B004E0ZLZQ

In [ ]:
import pandas as pd
from sentence_transformers import util

def print_search_results(search_results):
    results_list = []
    for asin, scores in search_results.items():
        if scores:
            results_list.append((asin, scores[0][1]))
    results_list.sort(key=lambda x: x[1], reverse=True)
    print("Search Results (Sorted by Similarity):")
    for i, (asin, score) in enumerate(results_list):
        print(f"{i+1}. ASIN: {asin}, Similarity Score: {score:.4f}")


In [ ]:
print_search_results(search_results)

Search Results (Sorted by Similarity):
1. ASIN: B001CK9760, Similarity Score: 0.5246
2. ASIN: B00WUNE7FE, Similarity Score: 0.5193
3. ASIN: B01N6UAWTY, Similarity Score: 0.5134
4. ASIN: B0015BYKGI, Similarity Score: 0.5044
5. ASIN: B00E9262IE, Similarity Score: 0.4975
6. ASIN: B00XJRS5AS, Similarity Score: 0.4795
7. ASIN: B000YA0A0Q, Similarity Score: 0.4736
8. ASIN: B00E9262YS, Similarity Score: 0.4720
9. ASIN: B01N14AT73, Similarity Score: 0.4716
10. ASIN: B004E0ZLZQ, Similarity Score: 0.4642
11. ASIN: B002LARRDA, Similarity Score: 0.4599
12. ASIN: B01N4ONWV3, Similarity Score: 0.4582
13. ASIN: B00GOOWRR0, Similarity Score: 0.4549
14. ASIN: B07PXGDCZJ, Similarity Score: 0.4521
15. ASIN: B008EQTHXS, Similarity Score: 0.4514
16. ASIN: B001M210V8, Similarity Score: 0.4498
17. ASIN: B00WU35CKS, Similarity Score: 0.4479
18. ASIN: B00WUTHOQ2, Similarity Score: 0.4452
19. ASIN: B00014B4OQ, Similarity Score: 0.4440
20. ASIN: B00WUFWCE0, Similarity Score: 0.4413
21. ASIN: B019MBSRQQ, Similari

In [ ]:
#metadata_df where parent asin =B0058NXWPY

metadata_df[metadata_df['parent_asin'] == 'B01N6UAWTY']

,parent_asin,category3,title,description,average_rating,processed_title,processed_description,combined_text
11761,B01N6UAWTY,Headphones,Libratone Lightning in-Ear Active Noise Cancel...,['Product Description'\n 'Libratone Q Adapt In...,3.6,libratone lightning ear active noise cancellin...,product description libratone adapt ear decide...,Libratone Lightning in-Ear Active Noise Cancel...
